### Questions

* Given two percussion loops, can we compute the offsets at which they can be mixed to yield the most phase, rhythmic and musical sum?
* As one loop slides over the other, doing a mix at each time step, which one of these mixes is the best?
* What does a mixing train-wreck look like?


* If we use synthesized audio loops at a given tempo, then a convolutional or denoising autoencoder to generate features, like onsets or beats, how good are those features versus say SuperFlux?
    


In [1]:
import os
import random
import math
import numpy as np
import librosa

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
kick_sample_path = "/Users/iorife/github/Investigaciones/Drums/Kick/Kick3kDeep.wav"
#kick_sample_path = "/Users/iorife/github/Investigaciones/Drums/Kick/Kick 90s 1.wav"

kick_sample, sr = librosa.load(kick_sample_path, mono=True, sr=48000)
print("Len of samples: " + str(len(kick_sample)))
print("Sample rate: " + str(sr))
y = createFourOnTheFloor(bpm=128, nb_beats=32, sr=sr, sample=kick_sample)
librosa.output.write_wav("4-on-floor-output.wav", y, sr)

plt.figure(figsize=(12, 6))
librosa.display.waveplot(y=y, sr=sr)

In [ ]:
remix_analysis = ['meta', 'id', 'md5',                                 # administrative
                  'duration', 'loudness',                              # singleton props of file
                  'time_signature', 'time_signature_confidence',       # time, key sigs + their confidence  
                  'key', 'key_confidence',
                  'tempo', 'tempo_confidence',                         # tempo + confidence
                  'bars', 'beats', 'sections', 'tatums', 'segments']   # remixing meat

librosa_remix = {}

# analysis params! no hardcoding!
hop_length = 32
fftsize = 1024

print("Hopsize in ms: " + str(float(hop_length)/float(sr)))
print("FFTsize in ms: " + str(float(fftsize)/float(sr)))


#-----------------------------------------------------------------------------------------------------------
# duration
librosa_remix['duration'] = librosa.get_duration(y=y, sr=sr)

#-----------------------------------------------------------------------------------------------------------
# loudness
linear_rms = np.sqrt(np.mean(np.square(y)))
dB_rms = 20 * np.log10((1e-20 + linear_rms))
librosa_remix['loudness'] = dB_rms

#-----------------------------------------------------------------------------------------------------------
# percussive - harmonic separation
D = librosa.stft(y, hop_length=hop_length)
D_harm, D_perc = librosa.decompose.hpss(D)
y_perc = librosa.istft(D_perc, hop_length=hop_length)
y_perc = y_perc / np.max(np.abs(y_perc), axis=0)

print("Len y:      " + str(len(y)))
print("Len y_perc: " + str(len(y_perc)))
plt.figure(figsize=(12, 6))
librosa.display.waveplot(y=y_perc, sr=sr)

# n = len(y)
# y_pad = librosa.util.fix_length(y, n + fftsize // 2)
# D = librosa.stft(y_pad, n_fft=n_fft)
# y_out = librosa.util.fix_length(librosa.istft(D), n)
# np.max(np.abs(y - y_out))

# y = y_perc

#-----------------------------------------------------------------------------------------------------------
# tempo
librosa_tempo, librosa_beat_frames = librosa.beat.beat_track(y, sr, hop_length=hop_length)
librosa_remix['tempo'] = librosa_tempo
librosa_remix['tempo_confidence'] = 1.0
print "librosa tempo is: ", librosa_remix['tempo']

#-----------------------------------------------------------------------------------------------------------
# key
# Longer FFT window to better resolve low frequencies
C = librosa.feature.chromagram(y=y, sr=sr, n_fft=fftsize, hop_length=hop_length)
p_classes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
p_class_counts = []

# let's find the average chroma value
mean = C.mean()
for p_class_name, values in zip(p_classes, C):
    count = (values > mean).sum()
    p_class_counts.append(count)
    
key_index = p_class_counts.index(max(p_class_counts))
key = p_classes[key_index]

# map key into echonest format [C, C#, D, Eb, E, F, F#, G, Ab, A, Bb, B]
librosa_remix['key'] = (key_index - 3) % 12
librosa_remix['key_confidence'] = 1.0
print "librosa key is: ", key, key_index
print "mapped -> echonest key is: ", librosa_remix['key'] 


#-----------------------------------------------------------------------------------------------------------
# librosa beats
librosa_beats = librosa.frames_to_time(librosa_beat_frames, sr, hop_length=hop_length)

# plot BEAT distributions!
print("Num beats: " + str(len(librosa_beats)))
print("Beat times: " + str(librosa_beats))

plt.figure(figsize=(12, 6))
x = np.diff(librosa_beats)
print(x)
bins = np.linspace(0.4, .5, 200)
plt.hist(x, bins, alpha=0.5, label='librosa diff start')
plt.legend(loc='upper right')
plt.show()

## Time Signature experiments

In [ ]:
import numpy.linalg as la    # import numpy linear algebra module
import scipy.spatial as spt  # import scipy spatial module

#-----------------------------------------------------------------------------------------------------------
def compute_squared_EDM_method1(X):
    
    # determine dimensions of data matrix X
    m,n = X.shape
    
    # initialize squared EDM D
    D = np.zeros((n,n))
    
    # iterate over upper triangle of D
    for i in range(n):
        for j in range(i+1,n):
            D[i,j] = la.norm(X[:,i] - X[:,j])**2
            D[j,i] = D[i,j]
    return D

#-----------------------------------------------------------------------------------------------------------
def compute_squared_EDM_method4(X):
    
    # determine dimensions of data matrix X
    m,n = X.shape
    
    # determine dimensions of data matrix X m,n = X.shape
    # compute Gram matrix G
    G = np.dot(X.T, X)
    
    # compute matrix H
    H = np.tile(np.diag(G), (n,1))
    return H + H.T - 2*G

#-----------------------------------------------------------------------------------------------------------
def compute_squared_EDM_method5(X):
    V = spt.distance.pdist(X.T, 'sqeuclidean')
    return spt.distance.squareform(V)


In [ ]:
# L == window length == (1/32) the duration a beat
#L = int( np.median(np.diff(librosa_beats)) * sr * (1.0/32.0))
tempo = 123.0

sample_rate = 44100
L = int( ( 60.0 / tempo ) * sample_rate * (1.0 / 32.0) )

# H == hop_length == (1/2) L
H = int(L * 0.5)

print "Window length: " + str(int(L)) + "  Hop length: " + str(int(H))

# Maximum bar length == 60 / 60  bpm * 12 beats = 12 seconds (12/4 time @ 60 bpm)
# Minimum bar length == 60 / 200 bpm * 02 beats = .6 seconds (2/4 time @ 200 bpm)

# start of first beat and read in N=4 seconds
# first_beat = librosa_beats[0]
first_beat = .985 # IO HAVOC --- OVERRIDE!!!
first_beat = .127

filenameAlt = "./test-audio/Bentos_RRReshape_snippet.wav"
yy, sr = librosa.load(filenameAlt, sr=None, offset=first_beat, duration=8.0)
D = librosa.stft(yy, n_fft=L, hop_length=H)
S, P = librosa.magphase(D)

# Cosine similarity (dot product)
# ASMatrix = np.dot(S.T, S)

# Squared euclidean 
ASMatrix = compute_squared_EDM_method5(S)
print "Similarity matrix shape: " + str(ASMatrix.shape)

# plot
plt.figure(figsize = (10, 10))
plt.pcolor(ASMatrix, cmap=plt.cm.ocean)
plt.colorbar()
plt.axes().set_aspect(1.0/plt.axes().get_data_ratio())
plt.show()

print np.argmax(ASMatrix, axis=1)

In [ ]:
#

def grabDiag(a, offset):
    altdiag = np.diag(np.rot90(a), k=offset)
    return altdiag[::-1] 

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

def computeSqrtSumSq(components):
#   print len(components)
    retval = np.sqrt(np.sum(np.square(components) / len(components), axis=0))
    return retval

# print grabDiag(a, 0)
# print grabDiag(a, 1)
# print grabDiag(a, 2)
# print grabDiag(a, 3)
# print grabDiag(a, 4)
# print grabDiag(a, 5)
# print grabDiag(a, 6)

# iterate through all bar lengths (i.e. num beats in Bar)
# components/beat is fixed at 1:64, so we're really wondering 
# components per bar, assuming that we start on the first beat (no inter-beat offset)
# and with the prior that the beats make up bars and the highest similarity is on the beat

SM = []
for beatsInBar in xrange(2, 13):
    componentsInBar = beatsInBar * 64
    print "Beats in bar: " + str(beatsInBar) + "  Components in bar: "+ str(componentsInBar)
    
    SCS_sum = 0
    SIS_sum = 0
    
    eqBar = componentsInBar
    eqR = 0
    eqSc = 0
    eqSi = 0
    
    # IO HAVOC -- what are we doing here, we need to grab diags on componentsInBar * j (on the beat components)
    # how many diagonals do we have?? depends on the len
    numDiags = ASMatrix.shape[0] / componentsInBar
    print "numDiags: " + str(numDiags)
    for diag in xrange(0, numDiags):
        
        currDiag = grabDiag(ASMatrix, componentsInBar + diag * componentsInBar)
        print  "Diag: " + str(diag) + " @ xpos - component #: " + str(componentsInBar + diag * componentsInBar)
        
        # split array to compute similarity: SCS (complete bars) and SIS (incomplete bars)
        splitarrs = [np.array(chunk) for chunk in chunker(currDiag, componentsInBar)]

        # save off the length of the        
        eqR = len(splitarrs[-1])
        
        # compute per diagonal SCS & SIS
        for arr in splitarrs[:-1]:
            temp = computeSqrtSumSq(arr)
            print "SCS len: " + str(len(arr)) + "  SCS: " + str(temp)
            SCS_sum += temp
            eqSc += 1
        else:
            temp = computeSqrtSumSq(splitarrs[-1]) 
            print "SIS len: " + str(len(splitarrs[-1])) + "  SIS: " + str(temp)
            SIS_sum += temp
            eqSi += 1
        
        print "  "
        
        peaks = np.zeros(len(currDiag))
        mx = np.max(currDiag)
        # make a mask to track componentsInBar components
        for i in xrange(0, len(currDiag)):
            if i % componentsInBar == 0:
                peaks[i] = mx

        print  "Diag: " + str(diag) + " @ xpos - component #: " + str(componentsInBar + diag * componentsInBar)
        x1 = np.linspace(0, len(currDiag), len(currDiag), endpoint=True)

        fig = plt.figure(figsize = (20, 7))
        plt.plot(x1, currDiag)
        plt.plot(x1, peaks, 'o-', label='peaks')

        plt.axes().set_autoscale_on(True)
        plt.axes().autoscale_view(True, True, False)
        plt.axes().set_ylabel(str(diag) + "  " + str(componentsInBar))
        
        
        
    
    print "Bar: " + str(eqBar) + "  r: " + str(eqR) + "  Sc: " + str(eqSc) + "  Si: " +  str(eqSi)
    SM.append((beatsInBar*SCS_sum +  eqR*SIS_sum)/( (eqBar*eqSc) +  (eqR*eqSi)))
#     SM.append((beatsInBar * SCS_sum + eqR * SIS_sum)/ 1.0 )

    print "SCS sum: " + str(SCS_sum) + "  SIS_sum: " + str(SIS_sum)
    print "\n"
print SM
    
    
# beatsInBar = 2
# componentsInBar = beatsInBar * 64    
# diagNum = 0   
# diagToPlot = grabDiag(ASMatrix, diagNum)
# print "ASMatrix diagonal #" + str(diagNum) + " has length: " + str(len(diagToPlot))

# # split array to compute similarity measures: 
# # SCS (complete bars) and SIS (incomplete bars)
# splitarrs = []
# for chunk in chunker(diagToPlot, componentsInBar):
#     splitarrs.append(np.array(chunk))

# # s = np.array(splitarrs)
# # for x in splitarrs:
# #     print len(x)

In [ ]:
# plot
plt.figure(figsize = (10, 10))
plt.plot(SM)
plt.axes().set_aspect(1.0/plt.axes().get_data_ratio())
plt.show()

In [ ]:
# plot
plt.figure(figsize = (10, 10))
plt.plot(yy)
plt.axes().set_aspect(1.0/plt.axes().get_data_ratio())
plt.show()

In [ ]:
############################################################
# iroro zoom in on one "beatsInBar" hypothesis at a time
############################################################

beatsInBar = 2
componentsInBar = beatsInBar * 64  

print "Beats in bar: " + str(beatsInBar) + "  Components in bar: "+ str(componentsInBar)
    
    
# IO HAVOC -- what are we doing here, we need to grab diags on componentsInBar * j (on the beat components)
# how many diagonals do we have?? depends on the len
numDiags = ASMatrix.shape[0] / (componentsInBar)
print "numDiags: " + str(numDiags)
for diag in xrange(0, numDiags):
    currDiag = grabDiag(ASMatrix, componentsInBar + diag * componentsInBar)
    peaks = np.zeros(len(currDiag))

    mx = 20000# np.max(currDiag)
    # make a mask to track componentsInBar components
    for i in xrange(0, len(currDiag)):
        if i % componentsInBar == 0:
            peaks[i] = mx
            
    print  "Diag: " + str(diag) + " @ xpos - component #: " + str(componentsInBar + diag * componentsInBar)
    x1 = np.linspace(0, len(currDiag), len(currDiag), endpoint=True)

    fig = plt.figure(figsize = (20, 7))
    plt.plot(x1, currDiag)
    plt.plot(x1, peaks, 'o-', label='peaks')
    
    plt.axes().set_autoscale_on(True)
    plt.axes().autoscale_view(True, True, False)
    plt.axes().set_ylabel(str(diag) + "  " + str(componentsInBar))

In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

beatsInBar = 2
componentsInBar = beatsInBar * 64    
diagNum = 0   
currDiag = grabDiag(ASMatrix, diagNum)
print "ASMatrix diagonal #" + str(diagNum) + " has length: " + str(len(currDiag))


# split array to compute similarity measures: 
# SCS (complete bars) and SIS (incomplete bars)
print len(currDiag), componentsInBar
splitarrs = []
for chunk in chunker(currDiag, componentsInBar):
    splitarrs.append(np.array(chunk))

s = np.array(splitarrs)
for x in splitarrs:
    print len(x)

# fig = plt.figure(figsize = (20, 7))
# x1 = np.linspace(0, len(diagToPlot), len(diagToPlot), endpoint=True)
# plt.plot(x1, diagToPlot)
# plt.axes().set_autoscale_on(True)
# plt.axes().autoscale_view(True, True, False)
# plt.axes().set_ylabel('diag') 
# plt.axes().fill_between(x1, diagToPlot, 0, where=x1 < componentsInBar, facecolor='green', interpolate=True)

# Random

In [ ]:
%%latex
\begin{align}
{C} = \textrm{Components in Bar}\\
SCS = \sqrt{ \frac{\displaystyle\sum_{i=1}^{C} G_i^2}{C} } \\
\\
SIS = \sqrt{ \frac{\displaystyle\sum_{i=1}^{r} P_i^2 }{r} }
\end{align}

In [ ]:
amp = 20 * np.log10(0.2)
print amp